# Imports

In [1]:
import selenium
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
#import numpy as np
import csv

In [2]:
import json
import os.path as osp
import platform
import time
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.proxy import Proxy, ProxyType

# Browser

In [3]:
browser = webdriver.Chrome(executable_path="chromedriver")
browser.maximize_window()
sleep(1)

# Extract shop information function

In [29]:
def extract_shop_info_mxb(shop_link):
    browser.get(shop_link)
    sleep(5) # slow to sign in
    
    # scroll all page by Chiu
    total_height = int(browser.execute_script("return document.body.scrollHeight"))
    for i in range(1, total_height, 100):
        browser.execute_script("window.scrollTo(0, {});".format(i))
    
    # parse html doc by Chiu
    soup = BeautifulSoup(browser.page_source, "html.parser")

    # initialize a dictionary to store information of products
    shop_info = {
            "rating": 0, # int (1 - 5)
            "num rating": "", # Number of rating (int)
            "quantity of products": 0, # 
            "response rate": 0, # float  (%)
            "response time": "", # str: (trong vai gio) 
            "followers" : -1, # Number of followers (int)
            "vouchers": -1, # Number of vouchers (int)
            "mall": -1, # 0 / 1: 0 for no, 1 for yes
            "cancel order rate": 0, # -1 for N/A, otherwise rate (0 - 1)
        }
    
    seller_overview_section = soup.find_all("div", {"class": "section-seller-overview__item-text-value"}) 

    # extract quantity of products
    quantity_products_label = seller_overview_section[0].text
    if quantity_products_label[-1] == "r": # tr (million)
        quantity_products_label = int(float(seller_overview_section[0].text[:-2].replace(",", ".")) * 1000000)
    elif quantity_products_label[-1] == "k": # k (thousand)
        quantity_products_label = int(float(seller_overview_section[0].text[:-1].replace(",", ".")) * 1000)
    else:
        quantity_products_label = int(seller_overview_section[0].text)
    shop_info["quantity of products"] = quantity_products_label

    # extract number of followers
    number_of_followers_label = seller_overview_section[1].text
    if number_of_followers_label[-1] == "r": # tr (million)
        number_of_followers_label = int(float(seller_overview_section[1].text[:-2].replace(",", ".")) * 1000000)
    elif number_of_followers_label[-1] == "k": # k (thousand)
        number_of_followers_label = int(float(seller_overview_section[1].text[:-1].replace(",", ".")) * 1000)
    else:
        number_of_followers_label = int(seller_overview_section[1].text)
    shop_info["followers"] = number_of_followers_label

    # extract rating
    rating_label = float(seller_overview_section[3].text.split()[0])
    shop_info["rating"] = rating_label

    # extract number of rating
    number_of_rating_label = seller_overview_section[3].text.split()[1][1:]
    if number_of_rating_label[-1] == "r": # tr (million)
        number_of_rating_label = int(float(seller_overview_section[3].text.split()[1][1:-2].replace(",", ".")) * 1000000)
    elif number_of_rating_label[-1] == "k": # k (thousand)
        number_of_rating_label = int(float(seller_overview_section[3].text.split()[1][1:-1].replace(",", ".")) * 1000)
    else:
        number_of_rating_label = int(seller_overview_section[3].text.split()[1][1:])
        
    shop_info["num rating"] = number_of_rating_label

    # extract response rate
    response_rate_label = float(seller_overview_section[4].text.split()[0][:-1]) / 100
    shop_info["response rate"] = response_rate_label

    # extract response time
    response_time_label = seller_overview_section[4].text.split("%")[1:][0].strip()
    shop_info["response time"] = response_time_label
    
    # extract number of vouchers
    try:
        vouchers_label = soup.find("div", {"class" : "shop-page__vouchers"})
        num_of_vouchers = len(vouchers_label.find_all("li", {"class" : "image-carousel__item"}))
    except:
        num_of_vouchers = 0
    shop_info["vouchers"] = num_of_vouchers

    # extract mall or not
    try:
        mall_label = soup.find("div", {"class" : "official-shop-new-badge"})
        if mall_label:
            mall_label = 1
    except:
        mall_label = 0
    shop_info["mall"] = mall_label

    # extract cancel orders rate
    try:
        cancel_orders_rate_label = seller_overview_section[6].text
        if cancel_orders_rate_label:
            cancel_orders_rate_label = float(cancel_orders_rate_label[:-1].replace(",", ".")) / 100
    except:
        cancel_orders_rate_label = -1
    shop_info["cancel order rate"] = cancel_orders_rate_label

    return shop_info


In [23]:
test_links = \
["https://shopee.vn/candlesvn", 
 "https://shopee.vn/coolmate.vn",
 "https://shopee.vn/larley.store", 
 "https://shopee.vn/aromastory.official",
 "https://shopee.vn/apple_flagship_store",
 "https://shopee.vn/xiaozhainv.vn",
 "https://shopee.vn/banhran2020"
]
# 1: Shopee Mall: Candles (clothes)
# 2: Shopee Mall: Coolmate (Clothes)
# 3: Shopee: Larley store (clothes)
# 4: Shopee: Aromastory (Candles & Aroma)
# 5: Shopee Mall: Apple Flagship Store
# 6: Shopee Mall: Shop Taobao (Clothes)
# 7: Shopee: Bán khẩu trang

In [30]:
for i, link in enumerate(test_links):
    print("Product {}".format(i + 1))
    print(extract_shop_info_mxb(link))
    print("\n")

Product 1
{'rating': 4.9, 'num rating': 8000, 'quantity of products': 113, 'response rate': 1.0, 'response time': '(trong vài giờ)', 'followers': 55400, 'vouchers': 1, 'mall': 1, 'cancel order rate': -1}


Product 2
{'rating': 4.9, 'num rating': 337800, 'quantity of products': 485, 'response rate': 0.97, 'response time': '(trong vài giờ)', 'followers': 1100000, 'vouchers': 3, 'mall': 1, 'cancel order rate': -1}


Product 3
{'rating': 4.8, 'num rating': 16900, 'quantity of products': 37, 'response rate': 0.96, 'response time': '(trong vài giờ)', 'followers': 45000, 'vouchers': 1, 'mall': None, 'cancel order rate': -1}


Product 4
{'rating': 4.9, 'num rating': 5500, 'quantity of products': 142, 'response rate': 0.98, 'response time': '(trong vài giờ)', 'followers': 20000, 'vouchers': 1, 'mall': None, 'cancel order rate': -1}


Product 5
{'rating': 4.9, 'num rating': 45300, 'quantity of products': 309, 'response rate': 0.96, 'response time': '(trong vài giờ)', 'followers': 513100, 'vouche